In [1]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
%matplotlib inline
import pandas as pd
import numpy as np
import math
from datetime import datetime, date
from scipy.optimize import minimize
!pip install quantlib > dev\null
import QuantLib as ql

In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
import pickle

with open("/content/drive/MyDrive/Colab Notebooks/sabr/msft_calls.pickle", 'rb') as msft_file:
  msft_calls = pickle.load(msft_file)
with open("/content/drive/MyDrive/Colab Notebooks/sabr/msft_puts.pickle", 'rb') as msft_file:
  msft_puts = pickle.load(msft_file)
with open("/content/drive/MyDrive/Colab Notebooks/sabr/expirations.pickle", 'rb') as msft_file:
  expirations = pickle.load(msft_file)

msft_stock_price = 294.3900146484375
day = '2022-03-16'
maturities = []
for expiration in expirations:
  maturities.append((datetime.strptime(expiration, '%Y-%m-%d') 
                    - datetime.strptime(day, '%Y-%m-%d') if day 
                     else datetime.today()).days / 256 )
maturities = np.array(maturities)

In [ ]:
from math import pow, sqrt , log
import numpy as np
from scipy.optimize import root, least_squares, differential_evolution, basinhopping
from matplotlib import pyplot as plt,cm
from pandas import DataFrame
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import mean_squared_error as MSE

In [ ]:
day_count = ql.Actual365Fixed() 
calendar = ql.UnitedStates(1)
calculation_date = ql.Date(16, 3, 2022)

spot = msft_stock_price  
ql.Settings.instance().evaluationDate = calculation_date          

risk_free_rate = 0.01
dividend_rate = 0.0

yield_ts = ql.YieldTermStructureHandle(                
    ql.FlatForward(calculation_date, risk_free_rate, day_count))
dividend_ts = ql.YieldTermStructureHandle(
    ql.FlatForward(calculation_date, dividend_rate, day_count))

'Following is a sample grid of volatilities for different expiration and strikes.'

In [ ]:
strikes = msft_calls[0].strike
for row in msft_calls:
  strikes = np.intersect1d(strikes, row.strike)
set_strikes = set(strikes)
print(len(strikes))

16


In [ ]:
data = []
for row in msft_calls:
  vols = []
  for vol, strike in zip(row.impliedVolatility, row.strike):
    if strike in set_strikes:
      vols.append(vol)
  data.append(vols)

In [ ]:
data = np.array(data)
data.shape

(17, 16)

In [ ]:
expiration_dates = []
for expir in expirations:
  expiration_dates.append(ql.Date(expir, '%Y-%m-%d'))
ttm = [ql.Period(m-calculation_date, ql.Days) for m in expiration_dates]

In [ ]:
class Heston(ql.HestonModel):
    def __init__(self, init_params=(0.02,0.2,0.5,0.1,0.01)):
        self.init_params=init_params
        v0, kappa, theta, sigma, rho = self.init_params
        process=ql.HestonProcess(yield_ts, dividend_ts, ql.QuoteHandle(ql.SimpleQuote(spot)), v0, kappa, theta, sigma, rho)
        super().__init__(process)
        self.engine = ql.AnalyticHestonEngine(self)
        self.vol_surf = ql.HestonBlackVolSurface(ql.HestonModelHandle(self), self.engine.Gatheral)
        
        self.build_helpers()
        
        
    def build_helpers(self, mat=ttm, K=strikes, vol=data):
        temp=[]
        for m,v in zip(mat,vol):
            for i,s in enumerate(K):
                temp.append( ql.HestonModelHelper(m, calendar, spot, s, ql.QuoteHandle(ql.SimpleQuote(v[i])), 
                                                  yield_ts, dividend_ts)  )
        for x in temp: x.setPricingEngine(self.engine)
        self.helpers=temp
        self.loss= [x.calibrationError() for x in self.helpers]
    
    def f_cost(self, params, norm=False):
        self.setParams( ql.Array(list(params)) )
        self.build_helpers()
        if norm == True:
            self.loss = np.sqrt(np.sum(self.loss))
        return self.loss
    
h = Heston()

In [ ]:
class methods:
    def qllm(self):    
        self.calibrate(self.helpers,ql.LevenbergMarquardt(1e-8, 1e-8, 1e-8),ql.EndCriteria(500, 300, 1.0e-8,1.0e-8, 1.0e-8))
    def scilm(self):    
        root(self.f_cost, self.init_params, method='lm')
    def ls(self):
        least_squares(self.f_cost, self.init_params)
    def de(self):
        bounds = [(0,1),(0.01,15), (0.01,1.), (-1,1), (0,1.0) ]
        differential_evolution(self.f_cost, bounds, args=(True,), maxiter=100)
    def bha(self):
        bounds = [(0,1),(0.01,15), (0.01,1.), (-1,1), (0,1.0) ]
        minimizer_kwargs = {"method": "L-BFGS-B", "bounds": bounds, "args":(True,) }
        basinhopping(self.f_cost ,self.init_params, niter=5, minimizer_kwargs=minimizer_kwargs, 
                            stepsize=0.005, interval=10)
    
method_dict = { 'QLLM': methods.qllm ,'SciLM': methods.scilm,'LS': methods.ls, 'DE': methods.de, 'BHA': methods.bha} 

In [ ]:
method_dict['QLLM'](h)
theta, kappa, sigma, rho, v0 = h.params()
print(f'theta={theta:0.4f}\nkappa={kappa:0.4f}\nsigma={sigma:0.4f}\nrho={rho:0.4f}\nv0={v0:0.4f}')

theta=0.1322
kappa=34.4396
sigma=8.0471
rho=-0.5558
v0=0.1113


# PLOTS

In [ ]:
X, Y = np.meshgrid(strikes, [m.length() for m in ttm])
Z = np.array([h.vol_surf.blackVol(m,s) 
            for m,s in zip(Y.reshape(1,-1)[0]/365, X.reshape(1,-1)[0])]).reshape(len(X),len(X[0]))

In [ ]:
mat_threshold = np.array(maturities) < 0.5
# mat_threshold = np.repeat(True, len(maturities))
fig = go.Figure(data=[go.Surface(z=Z[mat_threshold,:], 
                                 x=strikes, y=maturities[mat_threshold],
                                 contours = {'x': {'show': True, 'size': 0.001,
                                                   'start': msft_stock_price*0.999, 
                                                   'end': msft_stock_price*1.001,  
                                                   'color' : 'red'}})])

fig.update_layout(title='HESTON surface for call-option', autosize=False,
                  width=800, height=600,
                  margin=dict(l=30, r=30, b=30, t=60))

fig.update_layout(scene=dict(
                  xaxis_title="Strikes, $",
                  yaxis_title="Maturities, years",
                  zaxis_title="Implied volatilities"))

fig.show()

In [ ]:
mat_threshold = np.array(maturities) < 0.5
# mat_threshold = np.repeat(True, len(maturities))
fig = go.Figure(data=[go.Surface(z=data[mat_threshold,:], 
                                 x=strikes, y=maturities[mat_threshold],
                                 contours = {'x': {'show': True, 'size': 0.001,
                                                   'start': msft_stock_price*0.999, 
                                                   'end': msft_stock_price*1.001,  
                                                   'color' : 'red'}})])

fig.update_layout(title='Market', autosize=False,
                  width=800, height=600,
                  margin=dict(l=30, r=30, b=30, t=60))

fig.update_layout(scene=dict(
                  xaxis_title="Strikes, $",
                  yaxis_title="Maturities, years",
                  zaxis_title="Implied volatilities"))

fig.show()

In [ ]:
mat_threshold = np.array(maturities) < 1
# mat_threshold = np.repeat(True, len(maturities))
from plotly.subplots import make_subplots
fig = make_subplots(
    rows=1, cols=2, subplot_titles=('Market', 'Heston evaluated'),
    specs=[[{'type': 'surface'}, {'type': 'surface'}]])

fig.add_trace(go.Surface(z=data[mat_threshold,:], showscale=False,
                         x=strikes, y=maturities[mat_threshold],
                         contours = {'x': {'show': True, 'size': 0.001,
                                           'start': msft_stock_price*0.999, 
                                           'end': msft_stock_price*1.001,  
                                           'color' : 'red'}}),
              row=1, col=1)

fig.add_trace(go.Surface(z=Z[mat_threshold,:], showscale=False,
                         x=strikes, y=maturities[mat_threshold],
                         contours = {'x': {'show': True, 'size': 0.001,
                                           'start': msft_stock_price*0.999, 
                                           'end': msft_stock_price*1.001,  
                                           'color' : 'red'}}),
              row=1, col=2)

fig.update_layout(width=1100, height=600,
                  margin=dict(l=30, r=30, b=30, t=30))

fig.update_scenes(xaxis_title_text="Strikes, $",
                  yaxis_title_text="Maturities, y",
                  zaxis_title_text="IV")
fig.show()

In [ ]:
pred = Z[mat_threshold,:]
mrkt = data[mat_threshold,:]
MSE(mrkt, pred)

0.0007926421157199825